In [64]:
import pandas as pd
import  numpy as np
import ast
import json
import datetime
from fastai.imports import *
from fastai.tabular import *
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sklearn.preprocessing import MultiLabelBinarizer
import ast
pd.options.display.float_format = '{:.2f}'.format

train = pd.read_csv('C:/Users/idan.benaun/Documents/GitHub/Kaggle_TMDB_Box_Office_Prediction/Data/train.csv', parse_dates=['release_date'])
test = pd.read_csv('C:/Users/idan.benaun/Documents/GitHub/Kaggle_TMDB_Box_Office_Prediction/Data/test.csv', parse_dates=['release_date'])

dict_columns = ['belongs_to_collection', 'genres', 'production_companies', 'cast']
mlb = MultiLabelBinarizer()

In [65]:
def convert_string_to_list(strVal):
    if type(strVal) is not str:
        return  []
    else:
        return ast.literal_eval(strVal)
    
def format_dict_column_and_extract_names(strVal, col="name"):
    listOfItems = convert_string_to_list(strVal)
    
    return list(map(lambda x: x[col], listOfItems))

def extract_genres(df):
    df['genres'] = df['genres'].apply(format_dict_column_and_extract_names)
    
    return df.join(pd.DataFrame(mlb.fit_transform(df.pop('genres')),
                                columns=list(map(lambda x: 'genre_'+x, mlb.classes_)),index=df.index))

def extract_common(df, col='production_companies', limit=30):
    df[col] = df[col].apply(format_dict_column_and_extract_names)
    companiesCount = df[col].apply(pd.Series).stack().value_counts()
    companiesToKeep = companiesCount[companiesCount > limit].keys()
    
    #print("We'll keep the companies that appear more than 30 times:")
    #print(companiesToKeep)
    
    df[col] = df[col].apply(lambda x: list(filter(lambda i: i in companiesToKeep, x)))
    
    return df.join(pd.DataFrame(mlb.fit_transform(df.pop(col)),
                          columns=list(map(lambda x: col+'_'+x,mlb.classes_)),
                          index=df.index))

def add_x_length_column(df, col='cast'):
    castNames = df[col].apply(format_dict_column_and_extract_names)
    df[col + '_len'] = castNames.apply(lambda x: len(x))
    df.drop([col], axis=1, inplace=True)
    
    return df



df = extract_genres(train)
df = extract_common(df, col='production_companies', limit=30)
df = extract_common(df, col='production_countries', limit=20)
df = extract_common(df, col='spoken_languages', limit=10)
df = add_x_length_column(df, col='cast')
df = add_x_length_column(df, col='crew')
df = add_datepart(df, 'release_date')

df['belongs_to_collection_bool'] = np.where(df['belongs_to_collection'].isna(), 0, 1)
df.drop(['belongs_to_collection'], axis=1, inplace=True)

#df['last_year_revenue'] = df.groupby(['release_date_year'])['revenue'].mean().shift(1)

df.describe().T


,count,mean,std,min,25%,50%,75%,max
id,3000.00,1500.50,866.17,1.00,750.75,1500.50,2250.25,3000.00
budget,3000.00,22531334.11,37026086.41,0.00,0.00,8000000.00,29000000.00,380000000.00
popularity,3000.00,8.46,12.10,0.00,4.02,7.37,10.89,294.34
runtime,2998.00,107.86,22.09,0.00,94.00,104.00,118.00,338.00
revenue,3000.00,66725851.89,137532326.34,1.00,2379808.25,16807068.00,68919203.50,1519557910.00
genre_Action,3000.00,0.25,0.43,0.00,0.00,0.00,0.00,1.00
genre_Adventure,3000.00,0.15,0.35,0.00,0.00,0.00,0.00,1.00
genre_Animation,3000.00,0.05,0.21,0.00,0.00,0.00,0.00,1.00
genre_Comedy,3000.00,0.34,0.47,0.00,0.00,0.00,1.00,1.00
genre_Crime,3000.00,0.16,0.36,0.00,0.00,0.00,0.00,1.00
